# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
df = pd.read_csv("../Resources/Cities.csv")
cities_df=df[["city","lat","lng","max temp(f)","humidity(%)","cloudiness(%)",
              "wind speed(mph)", "country", "date"]]
cities_df.head()

,city,lat,lng,max temp(f),humidity(%),cloudiness(%),wind speed(mph),country,date
0,gornopravdinsk,60.05,69.90,268.76,98,93,2.34,RU,1604356382
1,east london,-33.02,27.91,292.55,84,69,6.07,ZA,1604356382
2,alta floresta,-9.88,-56.09,295.88,91,100,0.88,BR,1604356383
3,severobaykalsk,55.65,109.32,265.45,88,100,0.83,RU,1604356383
4,port elizabeth,-33.92,25.57,291.15,88,0,7.20,ZA,1604356383


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
gmaps.configure(api_key=g_key)

In [9]:
locations = cities_df[["lat", "lng"]].astype(float)
humidity = cities_df["humidity(%)"].astype(float)
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(
    locations,
    weights=humidity, 
    dissipating=False,
    max_intensity=100,
    point_radius = 1
)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [40]:
ideal_df = pd.DataFrame()
ideal_cities=[]

for index, row in cities_df.iterrows():
    if row["max temp(f)"]>= 73 and row["max temp(f)"]<=90 and row["wind speed(mph)"]<5 and row["cloudiness(%)"]==0 :
        ideal_cities.append(row["city"])

len(ideal_cities)

[268.76 292.55 295.88 265.45 291.15 294.26 304.15 285.81 294.15 266.15
 288.97 300.15 279.15 295.15 285.15 298.15 277.59 302.04 259.45 288.15
 299.26 292.59 261.68 272.79 296.63 293.15 275.15 278.15 295.53 298.92
 275.37 301.72 303.15 287.04 283.15 301.15 297.42 299.36 298.55 267.15
 295.71 299.49 296.82 297.46 287.06 296.15 272.77 291.36 275.1  300.76
 259.15 285.93 297.15 300.61 269.15 295.37 270.15 297.19 286.48 296.93
 282.15 276.15 290.93 292.15 286.15 267.43 262.15 289.96 271.15 302.15
 298.74 277.69 259.83 306.15 295.67 276.61 302.73 281.15 299.4  279.59
 275.93 274.75 294.82 297.01 294.57 294.59 277.96 286.45 299.58 290.15
 248.62 266.68 252.42 279.26 297.59 265.15 252.67 290.18 284.15 275.32
 303.71 279.38 290.54 300.05 302.09 300.07 264.84 280.64 299.15 289.15
 280.37 267.76 272.15 292.3  278.71 298.82 266.2  282.04 275.82 284.47
 272.11 286.35 278.51 296.2  298.41 294.52 296.48 277.66 294.76 298.6
 299.38 287.59 270.86 264.3  283.6  272.28 289.91 293.22 289.39 256.15
 298.86

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
